<h1>Pool.map</h1>
<h3> In <code>func.py</code> </h3>

```python
import time 
import os
def count(n):
    print("This process ID is: " + str(os.getpid()))
    for i in range(n):
        time.sleep(1)
        print("Now is: " + str((i+1)) + " s")
    return os.getpid()
```
<h3> In <code>main.py</code> </h3>

```python
import funcs
import os
import multiprocessing as mp
if __name__ == "__main__":
    pool = mp.Pool(processes=3)
    result = pool.map(funcs.count, [8, 4])
    print("Processes finished!")
```
<h3>The result is:</h3>
<p style='font-size:80%;background-color:rgb(220,220,220)'>This process ID is: 12128<br>
This process ID is: 5908<br>
Now is: 1 s<br>
Now is: 1 s<br>
Now is: 2 s<br>
Now is: 2 s<br>
Now is: 3 s<br>
Now is: 3 s<br>
Now is: 4 s<br>
Now is: 4 s<br>
Now is: 5 s<br>
Now is: 6 s<br>
Now is: 7 s<br>
Now is: 8 s<br>
Processes finished!</p>
<br>
<h3> Summary: </h3>
<p><code>Pool.map</code> will <span style="color:red">block</span> the main process until <span style="color:red">all</span> the processes in $pool$ are  finished</p>
<hr>

<h1>Pool.map_async</h1>
<p><code>Pool.map_async</code> returns a  class <a ref="https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.AsyncResult"><code>multiprocessing.pool.AsyncResult</code></a> which has 4 methods:</p>
<ol>
  <li>$get([timeout])$</li><br>
  <li>$wait([timeout])$</li><br>
  <li>$ready()$</li><br>
  <li>$successful()$</li><br>
</ol>
<p>Here are some examples: </p>

<h3> In <code>func.py</code> </h3>

```python
import time 
import os
def count(n):
    print("This process ID is: " + str(os.getpid()))
    for i in range(n):
        time.sleep(1)
        print("Now is: " + str((i+1)) + " s in " + str(os.getpid()))
    print("process\"" + str(os.getpid()) + "\" is finished")
    return os.getpid()
```
<h3> In <code>main.py</code> </h3>

```python
import funcs
import os
import time
import multiprocessing as mp

if __name__ == "__main__":
     pool = mp.Pool(processes=3)
    # Create 3 processes to execute "funcs.count" concurrently, and this does not block the main process
    result = pool.map_async(funcs.count, [1, 2, 3])
    print("Processing!")

    # "Main" start to sleep
    start = time.time()
    time.sleep(2)
    end = time.time()
    print("After sleep: %f s" % (end - start))

    # The main process will wait here until all processes are finished
    pid = result.get()
    print("pid: " + str(pid))

    print("Main finished!")
```
<h3>The result is:</h3>
<p style='font-size:100%;background-color:rgb(220,220,220)'>
Processing!<br>
This process ID is: 20820<br>
This process ID is: 17144<br>
This process ID is: 4312<br>
Now is: 1 s in 20820<br>
process"20820" is finished<br>
Now is: 1 s in 17144<br>
Now is: 1 s in 4312<br>
<br>
After sleep: 2.000302 s<br>
<br>
Now is: 2 s in 17144<br>
process"17144" is finished<br>
Now is: 2 s in 4312<br>
Now is: 3 s in 4312<br>
process"4312" is finished<br>
<br>
pid: [20820, 17144, 4312]<br>
Main finished!<br>
</p>
<br>


<h3> Summary: </h3>
<ol>
    <li><span style="color:red">All</span> processes in <code>pool</code> will shut down if the main process is finished.
    <li><code>Pool.map_async</code> will <span style="color:red">not</span> block the main process. The processes it creates will start to execute <b>concurrently</b> from this moment.</li>
    <li><code>.get()</code> <span style="color:red">block</span> the main process until it receives <span style="color:red">all</span> the results, and return the results from each process</li>
</ol>


所以用async的时候要小心，如果后面没有阻碍的东西，程序会结束，相当于这些子process也被终止了